In [20]:
#check login node
#import socket
#print('%s' % socket.gethostname())

cn0627


In [13]:
######These didn't work
##from rpy2.robjects.packages import importr
#utils = importr('utils')
#utils.install_packages('plink2R')
#####################################################
#cmd_0 ="Rscript R CMD INSTALL --library=/data/songy4/TWAS/plink2R-master/plink2R"
#shell_do(cmd_0)
####################################################
#!module load plink/2.0_alpha_1_final
#!module spider plink
#!devtools::install_github("gabraham/plink2R/plink2R")
#########################################################
#!devtools::install_github("carbocation/plink2R/plink2R", ref="carbocation-permit-r361")

Executing: Rscript R CMD INSTALL --library=/data/songy4/TWAS/plink2R-master/plink2R


In [9]:
#clone fusion_twas library before running this
!git clone https://github.com/gusevlab/fusion_twas.git

Cloning into 'fusion_twas'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 218 (delta 4), reused 1 (delta 0), pack-reused 204
Receiving objects: 100% (218/218), 4.12 MiB | 21.20 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [21]:
#open terminal in jupyter lab, run these in terminal 
!wget https://github.com/gabraham/plink2R/archive/master.zip
!unzip master.zip
#then start R in that same terminal by typing R and run below in R in terminal
install.packages('plink2R-master/plink2R/',repos=NULL)
#####Must install plink2R prior to calculate weights!!!!!

--2021-07-12 01:34:08--  https://github.com/gabraham/plink2R/archive/master.zip
Resolving dtn04-e0 (dtn04-e0)... 10.1.200.240
Connecting to dtn04-e0 (dtn04-e0)|10.1.200.240|:3128... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://codeload.github.com/gabraham/plink2R/zip/master [following]
--2021-07-12 01:34:08--  https://codeload.github.com/gabraham/plink2R/zip/master
Connecting to dtn04-e0 (dtn04-e0)|10.1.200.240|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: 'master.zip'

    [ <=>                                   ] 2,789,569   --.-K/s   in 0.09s   

2021-07-12 01:34:08 (28.5 MB/s) - 'master.zip' saved [2789569]

Archive:  master.zip
d74be015e8f54d662b96c6c2a52a614746f9030d
   creating: plink2R-master/
 extracting: plink2R-master/.gitignore  
  inflating: plink2R-master/README.md  
  inflating: plink2R-master/data.bed  
  inflating: plink2R-master/data.bim  
  inflating: plink2R-mas

In [1]:
import subprocess
import sys
import os
import shutil
import pandas as pd

In [2]:
def shell_do(command, log=False, return_log=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    res=subprocess.run(command.split(), stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return(res.stdout.decode('utf-8'))

In [3]:
#set paths
basedir = '/data/songy4/proteomics_196'
datadir = f'{basedir}/data_folder'
twasdir = f'/data/songy4/twas'
fusiondir = f'{twasdir}/fusion_twas'
geno_path = f'{datadir}/qc_proteomics196_csf_cardio'
###Add _case_hg19_lifted or _control_hg19_lifted
gene_list_path = f'{datadir}/PDBP_PPMI_proteomics_csf_cardio_gene_list.txt'
pheno_path = f'{datadir}/PDBP_PPMI_proteomics_196_csf_cardio_npx_expression'
###Add _case.txt or _control.txt
coord_path = f'{datadir}/PDBP_PPMI_proteomics_csf_cardio_coordinate.txt'
covar_path = f'{datadir}/covariates_csf_cardio'
###Add _case.txt or _control.txt


#geno_path = f'{datadir}/qc_genotypes_twas'
#gene_list_path = f'{datadir}/gene_list.txt'
#pheno_path = f'{datadir}/twas_expression_case.txt'
#coord_path = f'{datadir}/twas_coordinate.txt'
#covar_path = f'{datadir}/covariates_case.txt'
gcta = f'{fusiondir}/gcta_nr_robust'
plink = f'{fusiondir}/plink2R-master/plink2R'
gemma = f'gemma-0.98.3-linux-static'
fusion_ldref_basename = f'{fusiondir}/LDREF/1000G.EUR'
fusion_compute_weights_script = f'{fusiondir}/FUSION.compute_weights.R'

!mkdir --parents output/weights_csf_cardio_case
!mkdir --parents output/tmp_csf_cardio_case
!mkdir --parents output/weights_csf_cardio_control
!mkdir --parents output/tmp_csf_cardio_control

# Pipeline for TWAS run

In [4]:
# get gene list
##test with couple genes first before running all gene_list
gene_list_df = pd.read_csv(gene_list_path, sep='\t')
gene_list = list(gene_list_df.ID)
#gene_list = ['ACAN', 'ACP5', 'ACTA2'] # for testing
# gene_list = ['ENSG00000186092']
pheno_case = pd.read_csv(f'{pheno_path}_case.txt', sep='\t')
pheno_cont = pd.read_csv(f'{pheno_path}_control.txt', sep='\t')
coords = pd.read_csv(coord_path, sep='\t')
covar_case = pd.read_csv(f'{covar_path}_case.txt', sep='\t')
covar_cont = pd.read_csv(f'{covar_path}_control.txt', sep='\t')

In [5]:
# now put together pipeline

csf_cardio_case_compweights_swarmfile = f'{basedir}/csf_cardio_case_compute_weights.swarm'

with open(csf_cardio_case_compweights_swarmfile, 'w') as f:
    
    for gene in gene_list:
        OUT = f'output/tmp_csf_cardio_case/{gene}'
        FINAL_OUT = f'output/weights_csf_cardio_case/{gene}'
        #get chr start stop
        _chr = coords.loc[coords.ID == gene, 'X.Chr'].item()
        _start = coords.loc[coords.ID == gene, 'start'].item()-0.5e6
        _stop = coords.loc[coords.ID == gene, 'end'].item()+0.5e6

        if _start < 0:
            _start = 0
        
        _start = int(_start)
        _stop = int(_stop)
        
#         _temp_name = f'{gene}_temp'
#         _gene_temp = f'{tempdir}/{_temp_name}'
        
        # pheno per gene
#         _phenoname = f'{_gene_temp}.pheno'
        
        # write pheno file per gene
        pheno_case[['FID','IID', gene]].to_csv(f'{OUT}.pheno', sep='\t', header=False, index=False)
#         pheno[['FID','IID', gene]].to_csv(_phenoname, sep='\t', header=False, index=False)
        
        plink_cmd = f'\
plink --bfile {geno_path}_case_hg19_lifted \
--pheno {OUT}.pheno \
--keep {OUT}.pheno \
--chr {_chr} \
--from-bp {_start} \
--to-bp {_stop} \
--extract {fusion_ldref_basename}.{_chr}.bim \
--make-bed \
--out {OUT}'  

#        shell_do(plink_cmd, log=True, return_log=True)

        fusion_cmd = f'\
Rscript {fusion_compute_weights_script} \
--bfile {OUT} \
--tmp {OUT}.tmp \
--out {FINAL_OUT} \
--PATH_gemma {gemma} \
--PATH_plink plink \
--PATH_gcta {gcta} \
--covar {covar_path}_case.txt \
--verbose 2 \
--save_hsq \
--models top1,lasso,enet'
##include "blup,bslmm" to the --models in fusion_cmd for future     
# 
#        shell_do(fusion_cmd, log=True, return_log=True)

        f.write(f'{plink_cmd} && {fusion_cmd}\n')
    f.close()

In [6]:
# run swarm
swarm_cmd = f'swarm -f {csf_cardio_case_compweights_swarmfile} -g 2 -t 1 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm'
# shell_do(swarm_cmd)
print(swarm_cmd)
!{swarm_cmd}
##check swarm jobs state in hpc.nih.gov/dashboard/
##check .e, .o, and .log files of swarm jobs in swarm folder (same name as swarm jobid in hpc dashboard)
#swarm_cmd = f'swarm --devel -f {case_compweights_swarmfile} -g 2 -t 1 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm'

swarm -f /data/songy4/proteomics_196/case_compute_weights.swarm -g 2 -t 1 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink,GEMMA/0.96 --partition=norm
19477557


In [19]:
#check number of weights in output/weights folder 
###run this in terminal

#go to the weights folder
!cd /data/songy4/proteomics_196/output/weights_csf_cardio_case
#check how many files are in total
! ls | wc -l
#there are 357
#check how many .RDat file is in weights folder
!ls -lR ./*.RDat | wc -l
#there are 30 RDat files

34
ls: cannot access ./*.RDat: No such file or directory
0


TWAS

In [7]:
#open gene_chr_start_stop.tab -- this has chr and position info
chr_pos_df = pd.read_csv(coord_path, sep='\t')
#rearrange the columns 
chr_pos_df = chr_pos_df[['ID', 'X.Chr','start','end']]
#change column names in chr_pos_df
chr_pos_df.columns = ['ID', 'CHR', 'P0', 'P1']

chr_pos_df

,ID,CHR,P0,P1
0,NADK,1,1682671,1711896
1,NTproBNP,1,11917521,11918988
2,ZBTB17,1,16268364,16302627
3,PLA2G2A,1,20301925,20306932
4,HSPG2,1,22148738,22263790
...,...,...,...,...
338,SOD1,21,33031935,33041244
339,TFF3,21,43731777,43735761
340,CSTB,21,45192393,45196326
341,ITGB2,21,46305868,46351904


In [8]:
#make gene_path.csv
! ls -l ./output/weights_csf_cardio_case | grep -E 'ENSG|.RDat' | awk '{print $9", ./weights/"$9}' | sed 's/.wgt.RDat,/,/g' > ./data_folder/PDBP_PPMI_csf_cardio_gene_path_case.csv

## list files in weights folder, search lines with same pattern as 'ENSG'
## scan file line by line, splits each input line
## substitute .wgt.RDat, with , in all occurrences (/g means global replacement)

In [9]:
#import gene_path.csv 
path = pd.read_csv(f"{datadir}/PDBP_PPMI_csf_cardio_gene_path_case.csv", names=['ID', 'WGT'])
path.head

<bound method NDFrame.head of          ID                          WGT
0      ACP5      ./weights/ACP5.wgt.RDat
1      ART3      ./weights/ART3.wgt.RDat
2      CD46      ./weights/CD46.wgt.RDat
3      CDH1      ./weights/CDH1.wgt.RDat
4    CHI3L1    ./weights/CHI3L1.wgt.RDat
5      CST6      ./weights/CST6.wgt.RDat
6      CTSH      ./weights/CTSH.wgt.RDat
7     EPHB4     ./weights/EPHB4.wgt.RDat
8       FAS       ./weights/FAS.wgt.RDat
9    FCGR2A    ./weights/FCGR2A.wgt.RDat
10    FUCA1     ./weights/FUCA1.wgt.RDat
11     GLO1      ./weights/GLO1.wgt.RDat
12   IL1RL1    ./weights/IL1RL1.wgt.RDat
13     IL6R      ./weights/IL6R.wgt.RDat
14   LGALS3    ./weights/LGALS3.wgt.RDat
15   LILRB5    ./weights/LILRB5.wgt.RDat
16   NOTCH3    ./weights/NOTCH3.wgt.RDat
17     NRP1      ./weights/NRP1.wgt.RDat
18    PDCD6     ./weights/PDCD6.wgt.RDat
19   PDGFRB    ./weights/PDGFRB.wgt.RDat
20    PILRB     ./weights/PILRB.wgt.RDat
21   PM20D1    ./weights/PM20D1.wgt.RDat
22     PON2      ./weights/

In [34]:
#remove .hsq rows
#path = path[~path.WGT.str.contains(".hsq")]
#path.head
#save path as csv -- run only once for download
#path.to_csv(r'./data_folder/PDBP_PPMI_csf_cardio_gene_path_case.csv' ,index=False)

<bound method NDFrame.head of                     ID                                  WGT
2      ENSG00000000457   ./weights/ENSG00000000457.wgt.RDat
4      ENSG00000000460   ./weights/ENSG00000000460.wgt.RDat
7      ENSG00000000971   ./weights/ENSG00000000971.wgt.RDat
9      ENSG00000001036   ./weights/ENSG00000001036.wgt.RDat
11     ENSG00000001084   ./weights/ENSG00000001084.wgt.RDat
...                ...                                  ...
27485  ENSG00000272514   ./weights/ENSG00000272514.wgt.RDat
27489  ENSG00000272636   ./weights/ENSG00000272636.wgt.RDat
27494  ENSG00000273045   ./weights/ENSG00000273045.wgt.RDat
27501  ENSG00000273155   ./weights/ENSG00000273155.wgt.RDat
27503  ENSG00000273173   ./weights/ENSG00000273173.wgt.RDat

[9534 rows x 2 columns]>

In [10]:
#merge chr_pos_df and path
merged = chr_pos_df.merge(path, how='inner', on='ID')
merged.head

<bound method NDFrame.head of          ID  CHR         P0         P1                          WGT
0     FUCA1    1   24171567   24194784     ./weights/FUCA1.wgt.RDat
1      IL6R    1  154377669  154441926      ./weights/IL6R.wgt.RDat
2    FCGR2A    1  161475220  161493803    ./weights/FCGR2A.wgt.RDat
3      SELP    1  169558087  169599431      ./weights/SELP.wgt.RDat
4    CHI3L1    1  203148059  203155877    ./weights/CHI3L1.wgt.RDat
5    PM20D1    1  205797150  205819260    ./weights/PM20D1.wgt.RDat
6      CD46    1  207925402  207968858      ./weights/CD46.wgt.RDat
7      QPCT    2   37571717   37600465      ./weights/QPCT.wgt.RDat
8    IL1RL1    2  102927962  102968497    ./weights/IL1RL1.wgt.RDat
9      ART3    4   76932337   77033955      ./weights/ART3.wgt.RDat
10    PDCD6    5     271736     353971     ./weights/PDCD6.wgt.RDat
11   PDGFRB    5  149493400  149535435    ./weights/PDGFRB.wgt.RDat
12    TIMD4    5  156346293  156390266     ./weights/TIMD4.wgt.RDat
13     GLO1    6  

In [11]:
#check chr 
chr_list = merged['CHR'].unique().tolist()
chr_list

[1, 2, 4, 5, 6, 7, 10, 11, 13, 14, 15, 16, 19, 22]

In [12]:
#drop ./weights/ from WGT
merged['WGT'] = merged['WGT'].str.replace('./weights/', '')
merged.head

<ipython-input-12-0109c7f231a0>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  merged['WGT'] = merged['WGT'].str.replace('./weights/', '')


<bound method NDFrame.head of          ID  CHR         P0         P1                WGT
0     FUCA1    1   24171567   24194784     FUCA1.wgt.RDat
1      IL6R    1  154377669  154441926      IL6R.wgt.RDat
2    FCGR2A    1  161475220  161493803    FCGR2A.wgt.RDat
3      SELP    1  169558087  169599431      SELP.wgt.RDat
4    CHI3L1    1  203148059  203155877    CHI3L1.wgt.RDat
5    PM20D1    1  205797150  205819260    PM20D1.wgt.RDat
6      CD46    1  207925402  207968858      CD46.wgt.RDat
7      QPCT    2   37571717   37600465      QPCT.wgt.RDat
8    IL1RL1    2  102927962  102968497    IL1RL1.wgt.RDat
9      ART3    4   76932337   77033955      ART3.wgt.RDat
10    PDCD6    5     271736     353971     PDCD6.wgt.RDat
11   PDGFRB    5  149493400  149535435    PDGFRB.wgt.RDat
12    TIMD4    5  156346293  156390266     TIMD4.wgt.RDat
13     GLO1    6   38643701   38670917      GLO1.wgt.RDat
14     PON2    7   95034175   95064510      PON2.wgt.RDat
15    PILRB    7   99933737   99965356    

In [13]:
#rearrange columns and save the file as synapse.pos
output = merged[['WGT', 'ID', 'CHR', 'P1', 'P0']]
output.to_csv(r'./data_folder/PDBP_PPMI_csf_cardio_synapse_case.pos', index=False, sep="\t")

In [14]:
#set paths
sumstat_path = f'{datadir}/meta.txt'
weights_path = f'{datadir}/PDBP_PPMI_csf_cardio_synapse_case.pos'
weights_dir = f'{basedir}/output/weights_csf_cardio_case/'
fusion_ldref_basename = f'{fusiondir}/LDREF/1000G.EUR'
fusion_assoc_script = f'{fusiondir}/FUSION.assoc_test.R'

!mkdir --parents output/pd_csf_cardio_case

sumstat = pd.read_csv(sumstat_path, sep='\t')

##check if weights_dir should be f'{basedir}/output/weights'####

In [15]:
#fusion calculation -- take about 3 min each chromosome 
for i in range(1, 23):
    PD_OUT = f'output/pd_csf_cardio_case/PD.{i}.dat'
    fusion_cmd_i = f'\
    Rscript {fusion_assoc_script} \
    --sumstats {sumstat_path} \
    --weights {weights_path} \
    --weights_dir {weights_dir} \
    --ref_ld_chr {fusion_ldref_basename}. \
    --chr {i} \
    --out {PD_OUT}'
    shell_do(fusion_cmd_i)

Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/proteomics_196/data_folder/meta.txt --weights /data/songy4/proteomics_196/data_folder/PDBP_PPMI_csf_cardio_synapse_case.pos --weights_dir /data/songy4/proteomics_196/output/weights_csf_cardio_case/ --ref_ld_chr /data/songy4/twas/fusion_twas/LDREF/1000G.EUR. --chr 1 --out output/pd_csf_cardio_case/PD.1.dat
Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/proteomics_196/data_folder/meta.txt --weights /data/songy4/proteomics_196/data_folder/PDBP_PPMI_csf_cardio_synapse_case.pos --weights_dir /data/songy4/proteomics_196/output/weights_csf_cardio_case/ --ref_ld_chr /data/songy4/twas/fusion_twas/LDREF/1000G.EUR. --chr 2 --out output/pd_csf_cardio_case/PD.2.dat
Executing: Rscript /data/songy4/twas/fusion_twas/FUSION.assoc_test.R --sumstats /data/songy4/proteomics_196/data_folder/meta.txt --weights /data/songy4/proteomics_196/data_folder/PDBP_PPMI_csf_cardio_

# running list of questions
1. what about non-autosome transcripts? i.e. MT
2. how many snps in common between ref and geno?
3. how many snps per transcript should we expect (0-500?)


In [44]:
p_in = 'tmp/ENSG00000188976.tmp.cv'
f"gemma -miss 1 -maf 0 -r2 1 -rpace 1000 -wpace 1000 -bfile {p_in} -bslmm 2 -o ENSG00000188976"

'gemma -miss 1 -maf 0 -r2 1 -rpace 1000 -wpace 1000 -bfile tmp/ENSG00000188976.tmp.cv -bslmm 2 -o ENSG00000188976'